In [5]:
a=3
b=15
resultado=a*b
print("mi resultado es:",resultado)

mi resultado es: 45


# Introducción a openquake

## Instalación

- descarga [repositorio de openquake](https://github.com/gem/oq-engine)
- *vscode*
- **openquake**
- `a = 1 + 2`

In [ ]:
# Basic python modules
import numpy as np
import matplotlib.pyplot as plt
import os

# OQ modules
from openquake.hazardlib.gsim import get_available_gsims
from openquake.hazardlib.gsim.base import RuptureContext, SitesContext, DistancesContext
from openquake.hazardlib.imt import SA
from openquake.hazardlib import const

In [2]:
gmpe_list = get_available_gsims()

In [3]:
rctx = RuptureContext()
rctx.mag = np.array([8.8])
sctx = SitesContext()
sctx.vs30 = np.array([700.])
sctx.backarc = False
sctx.sids = np.array([0])
dctx = DistancesContext()
dctx.rrup = np.array([150])

In [4]:
periods = np.logspace(-2, 1, 30)

imts = [SA(period) for period in periods]

stddev_types = [const.StdDev.TOTAL]

In [5]:
means = np.zeros(len(imts))
means_plus_1sd = np.zeros(len(imts))
means_minus_1sd = np.zeros(len(imts))
stddevs = np.zeros(len(imts))

gmpes = [gmpe_list["MontalvaEtAl2017SInter"](),
         gmpe_list["ParkerEtAl2020SInter"]()]

In [6]:
def calculate_ground_motions(gmpes, imts, sctx, rctx, dctx, stddev_types):
    """
    Calculates the expected ground motion and uncertainty, organised by GMPE
    and intensity measure type (i.e. PGA, SA etc.), for a given rupture-site configuration    
    """
    results = {}
    nper = len(imts)
    for gmpe in gmpes:
        print("Running GMPE %s" % str(gmpe))
        results[str(gmpe)] = {"mean": np.zeros(nper),
                              "stddevs": np.zeros(nper),
                              "mean_plus_1sd": np.zeros(nper),
                              "mean_minus_1sd": np.zeros(nper)}
        for i, imt in enumerate(imts):
            try:
                mean, [stddev] = gmpe.get_mean_and_stddevs(
                    sctx, rctx, dctx, imt, stddev_types)
                results[gmpe]["mean"][i] = np.exp(mean)
                results[gmpe]["stddevs"][i] = stddev
                results[gmpe]["mean_plus_1sd"][i] = np.exp(mean + stddev)
                results[gmpe]["mean_minus_1sd"][i] = np.exp(mean - stddev)
            except KeyError:
                results[gmpe]["mean"][i] = np.nan
                results[gmpe]["stddevs"][i] = np.nan
                results[gmpe]["mean_plus_1sd"][i] = np.nan
                results[gmpe]["mean_minus_1sd"][i] = np.nan
    return results


In [7]:
results = calculate_ground_motions(gmpes, imts, sctx, rctx, dctx, stddev_types)

Running GMPE [MontalvaEtAl2017SInter]
Running GMPE [ParkerEtAl2020SInter]


In [8]:
for key in results.keys():
    print(str(key))
    print(results[key])

[MontalvaEtAl2017SInter]
{'mean': array([0.086734  , 0.08738247, 0.08803579, 0.08917644, 0.09522407,
       0.10131618, 0.1074393 , 0.11543564, 0.13022949, 0.14806517,
       0.17008116, 0.19548271, 0.22070644, 0.23282973, 0.21265885,
       0.1879996 , 0.15986412, 0.12855319, 0.10473855, 0.08659512,
       0.06558282, 0.04708044, 0.03700144, 0.0299099 , 0.02060181,
       0.01594505, 0.01215415, 0.00967288, 0.0089769 , 0.00838651]), 'stddevs': array([0.83844918, 0.84110491, 0.84376063, 0.84706472, 0.85692103,
       0.86677733, 0.87663364, 0.88760494, 0.90204425, 0.90901477,
       0.9079249 , 0.90338391, 0.88436829, 0.86310782, 0.84420147,
       0.83276501, 0.81203524, 0.80134624, 0.81017091, 0.80974758,
       0.8023014 , 0.79204019, 0.76837164, 0.75729074, 0.73362281,
       0.69128478, 0.67692496, 0.64379338, 0.62427024, 0.6024269 ]), 'mean_plus_1sd': array([0.20059645, 0.20263364, 0.20469152, 0.20802985, 0.22433799,
       0.24105459, 0.25815488, 0.28042829, 0.32096833, 0.367479